In [14]:
from utils import RNN, device, SampleMetroDataset, ForecastMetroDataset
import torch
from torch.utils.data import DataLoader, Dataset
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

# Nombre de stations utilisé
CLASSES = 5
#Longueur des séquences
LENGTH = 30
# Dimension de l'entrée (1 (in) ou 2 (in/out))
DIM_INPUT = 1
#Taille du batch
BATCH_SIZE = 32

PATH = "../../data/"


matrix_train, matrix_test = torch.load(open(PATH+"hzdataset.pch","rb"))
ds_train = SampleMetroDataset(matrix_train[:, :, :CLASSES, :DIM_INPUT], length=LENGTH)
ds_test = SampleMetroDataset(matrix_test[:, :, :CLASSES, :DIM_INPUT], length = LENGTH, stations_max = ds_train.stations_max)
data_train = DataLoader(ds_train,batch_size=BATCH_SIZE,shuffle=True)
data_test = DataLoader(ds_test, batch_size=BATCH_SIZE,shuffle=False)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
class State :
    def __init__ (self, model, optim):
        self.model = model
        self.optim = optim
        self.epoch, self.iteration = 0, 0

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [17]:
def accuracy(ypred, y):
    """
    Input
        @ypred  : tensor (batch, nbclass) contient raw logits -> sigmoid -> softmax -> argmax
        @y      : tensor (batch,) contient la vraie classe
    """
    ypred_ = torch.argmax(torch.softmax(ypred, dim=-1), dim=-1)
    accuracy = (ypred_==y).sum()/len(y)
    return accuracy

In [18]:
savepath = Path(f"exo2_{LENGTH}.plt")
HIDDEN = 40
lr = 0.005
NB_EPOCH = 50
if savepath.is_file():
    with savepath.open("rb") as fp :
        state = torch.load(fp)
else:
    model = RNN(in_dim=DIM_INPUT, out_dim=CLASSES, hidden=HIDDEN).to(device)
    optim = torch.optim.SGD(params=model.parameters(),lr=lr)
    state = State(model, optim)

loss = torch.nn.CrossEntropyLoss()

In [19]:
writer = SummaryWriter()
for epoch in range(state.epoch, NB_EPOCH):
    print(f'Epoch {epoch+1}')
    acc_train = []
    for x, y in data_train:
        x = torch.transpose(x, dim0=0, dim1=1)
        #y = y.type(torch.float32)
        bz = x.shape[1]
        x = x.to(device)
        h = state.model.init_h0(bz)
        h = state.model.forward(x, h)
        ypred = state.model.decode(h)[-1] # on prends le dernier ---- many to one
        acc_train.append(accuracy(ypred.detach(), y))
        state.optim.zero_grad()
        l = loss(ypred, y)
        l.backward()
        state.optim.step()
        state.iteration += 1
        writer.add_scalar('Loss/train', l, state.iteration)

    with savepath.open("wb") as fp :
        state.epoch = epoch + 1
        torch.save(state, fp)

    with torch.no_grad():
        acc_test = []
        los_test = []
        for x_test,y_test in data_test:
            x_test = torch.transpose(x_test, dim0=0, dim1=1)
            x_test = x_test.to(device)
            bz = x_test.shape[1]
            h_test = state.model.init_h0(bz)
            h_test = state.model.forward(x_test, h_test)
            ypred_test = state.model.decode(h_test)[-1]
            acc_test.append(accuracy(ypred_test, y_test))
            los_test.append(loss(ypred_test, y_test))
        writer.add_scalar('Loss/test', sum(los_test)/len(los_test), state.epoch)
    writer.add_scalar('Acc/train', sum(acc_train)/len(acc_train), state.epoch)
    writer.add_scalar('Acc/test', sum(acc_test)/len(acc_test), state.epoch)

writer.close()

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50


## Many to Many

In [20]:
LENGTH = 20
DIM_INPUT = 1
#Taille du batch
BATCH_SIZE = 32

ds_train = ForecastMetroDataset(
    matrix_train[:, :, :CLASSES, :DIM_INPUT], length=LENGTH)
ds_test = ForecastMetroDataset(
    matrix_test[:, :, :CLASSES, :DIM_INPUT], length=LENGTH, stations_max=ds_train.stations_max)
data_train = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True)
data_test = DataLoader(ds_test, batch_size=BATCH_SIZE, shuffle=False)

In [21]:
savepath = Path(f"exo2_{LENGTH}_forecast.plt")
HIDDEN = 40
lr = 0.005
NB_EPOCH = 200
if savepath.is_file():
    with savepath.open("rb") as fp :
        state = torch.load(fp)
else:
    model = RNN(in_dim=DIM_INPUT*CLASSES, out_dim=DIM_INPUT*CLASSES, hidden=HIDDEN).to(device) # forecast n from n -> out = in
    optim = torch.optim.SGD(params=model.parameters(),lr=lr)
    state = State(model, optim)

# MAE pas sensible aux outliers
#loss = torch.nn.L1Loss()
# sinon
loss = torch.nn.MSELoss()

In [22]:
writer = SummaryWriter()
for epoch in range(state.epoch, NB_EPOCH):
    print(f'Epoch {epoch+1}')
    for x, y in data_train:
        # x (batch, len, stations, d)
        x = torch.reshape(x, (x.size(1), x.size(0), -1)) # (len, batch, D)
        y = torch.reshape(y, (y.size(1), y.size(0), -1))
        #y = y.type(torch.float32)
        bz = x.shape[1]
        x = x.to(device)
        h = state.model.init_h0(bz) 
        h = state.model.forward(x, h)
        ypred = state.model.decode(h)
        ## sigmoid, output [0-1]
        ypred = torch.nn.Sigmoid()(ypred)
        state.optim.zero_grad()
        l = loss(ypred, y)
        l.backward()
        state.optim.step()
        state.iteration += 1
        writer.add_scalar('Loss/train', l, state.iteration)

    with savepath.open("wb") as fp :
        state.epoch = epoch + 1
        torch.save(state, fp)

    with torch.no_grad():
        los_test = []
        for x_test,y_test in data_test:
            x_test = torch.reshape(x_test, (x_test.size(1), x_test.size(0), -1)) # (len, batch, D)
            y_test = torch.reshape(y_test, (y_test.size(1), y_test.size(0), -1))
            x_test = x_test.to(device)
            bz = x_test.shape[1]
            h_test = state.model.init_h0(bz)
            h_test = state.model.forward(x_test, h_test)
            ypred_test = state.model.decode(h_test)
            #
            ypred_test = torch.nn.Sigmoid()(ypred_test)

            los_test.append(loss(ypred_test, y_test))
        writer.add_scalar('Loss/test', sum(los_test)/len(los_test), state.epoch)

writer.close()

Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110
Epoch 11

In [24]:
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 2625), started 0:00:06 ago. (Use '!kill 2625' to kill it.)